# LNR - Práctica 3
## Sesión 3: Validación de Clasificadores

Nombre:
- Guillermo Ferrando Muñoz

Ejercicio:

Ajustar los parámetros del clasificador propuesto para la tarea DETESTS empleando la
medida de evaluación F1, y empleando un esquema de validación cruzada. Entregar código
e informe con la explicación de la estrategia utilizada antes de la siguiente sesión.

##  Carga de datos

Cargamos los datos con el desbalanceo solucionado mediante SMOTE oversampling que obtuvimos en la sesión anterior de esta práctica. Para ello, leemos el documento "S3_P3.csv" que he generado, lo incluiré en la entrega de la tarea.

In [1]:
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn import svm
from sklearn.utils import shuffle
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

In [2]:
df=pd.read_csv("S3_P3.csv")
df=df.drop(columns=['Unnamed: 0']) #para eliminar una columna inútil
df.head(10)

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,stereotype
0,0.001318,0.003393,0.002340,-0.002296,0.004027,0.002442,0.000703,0.001406,-0.002482,0.000745,...,-0.001195,-0.000311,-0.000729,0.001500,-0.000136,-0.002057,0.004157,0.002924,0.003697,0
1,0.001163,0.002280,0.005396,-0.005706,0.003198,0.001721,0.002030,0.004187,-0.003248,0.007304,...,-0.002858,0.007483,-0.001480,0.005370,-0.002683,0.005325,-0.001532,0.000768,-0.002241,0
2,-0.001684,0.003351,-0.000300,0.000164,0.000150,-0.003695,0.001025,0.005774,0.001311,-0.000973,...,0.000300,0.000582,-0.004117,0.001826,0.002370,0.000677,-0.001318,-0.000238,0.002707,0
3,0.001581,-0.000432,0.002926,-0.001243,0.000831,-0.002557,0.003618,0.003611,0.000520,-0.000479,...,0.001939,-0.002411,-0.002368,0.001319,-0.000663,0.004117,-0.000327,0.003946,0.004706,0
4,-0.000496,-0.000016,0.007480,-0.002841,-0.001257,-0.001341,0.001828,0.003550,0.003153,0.001166,...,0.000466,0.005146,0.002355,0.002462,0.003463,0.002679,0.001108,-0.001922,0.000507,0
5,0.000717,0.001005,-0.000263,0.000679,0.001691,-0.003206,-0.002786,0.006954,-0.001253,0.001588,...,0.000191,-0.002558,-0.002527,0.007731,0.003119,-0.003920,0.001947,-0.004730,-0.002866,0
6,-0.004574,-0.001306,0.000020,-0.002135,-0.000803,0.000261,-0.000666,0.002697,0.000333,0.002623,...,-0.002448,0.001542,0.000938,0.005675,-0.000728,0.002071,-0.001132,0.002149,-0.001617,0
7,-0.007377,0.000183,0.004989,-0.006015,0.002102,-0.002705,-0.006664,0.009896,-0.009904,0.007932,...,0.008981,0.001597,0.000005,-0.006048,0.009836,0.006450,-0.005930,0.005494,0.005669,0
8,-0.000654,0.001425,0.000660,0.001815,-0.002990,-0.001975,0.000159,0.004599,-0.004364,0.000639,...,-0.003120,0.004801,-0.000113,0.003341,0.002509,0.002654,0.000550,0.001538,0.003378,0
9,0.004029,0.002029,0.004909,-0.000626,0.000693,-0.003865,0.001327,-0.008393,-0.000097,-0.005665,...,0.001387,0.005404,-0.005205,0.001415,0.006878,0.000329,-0.003058,-0.003888,0.002836,0


In [3]:
X = df.drop("stereotype", axis=1).values
y = df["stereotype"].values
#X_train, y_train = shuffle(X_train, y_train) #para barajar los datos

## Estrategias de validación

Vamos a ir implemetando las distintas estrategias de validación de modelos que se indican en el boletín de la práctica. Comenzamos partiendo los datos en para el entrenamiento y la prueba. El 25% de los datos será la parte de prueba.

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                          test_size=0.25, shuffle = True, random_state=42) #shuffle = True para barajar los datos

## Holdout

### Validación de SVM mediante Holdout

El primer método es **Holdout**, como decíamos antes, la parte de test el el 25%, que está entre en 1/3% y el 1/2% que se menciona en la explicación del boletín. El modelo a utilizar será **SVM**, ya que vimos en la anterior sesión que daba los mejores resultados, y para validar, haremos una única iteración y evaluaremos con la medida F1:

In [5]:
import time
from sklearn.metrics import f1_score, accuracy_score

tini = time.process_time()

clf = svm.SVC(kernel='linear', C=1, random_state=42).fit(X_train, y_train)
y_pred = clf.predict(X_test)

a=f1_score(y_test, y_pred, average=None)
print("F1", a)

a=f1_score(y_test, y_pred, average='macro') #Calculate metrics for each label, and find their unweighted mean. 
                                                #This does not take label imbalance into account.
print("macro F1", a)

a=f1_score(y_test, y_pred, average='micro') #Calculate metrics globally by counting the total true positives, 
                                                #false negatives and false positives.
print("micro F1", a)

print()
print('Métricas con validación cruzada: ')

macro = cross_val_score(clf, X_train, y_train, cv=5, scoring='f1_macro')
print("macro F1", macro)

micro = cross_val_score(clf, X_train, y_train, cv=5, scoring='f1_micro')
print("micro F1", micro)

print("Accuracy:", accuracy_score(y_test, y_pred, normalize=True))


tfin = time.process_time()
print()
print("Ha tardado", round(tfin-tini, 2), "segundos")

F1 [0.         0.66273264]
macro F1 0.33136631865637767
micro F1 0.4955872369314325

Métricas con validación cruzada: 
macro F1 [0.33433735 0.33383572 0.33383572 0.33383572 0.33408748]
micro F1 [0.50226244 0.50113122 0.50113122 0.50113122 0.50169875]

Ha tardado 14.39 segundos


Podemos observar que los resultados se obtienen muy rápido, pero no son buenos, ya que el kernel utilizado es lineal y el parámetro `C=1` no daba buenos resultados, según vimos en la anterior sesión. Por ello, vamos a aplicar **GridSearchCV** para intentar encontrar los mejores hiperparámetros.

### Modelo óptimo SVM según GridSearchCV

Vamos a probar con todos los kernels posibles que ofrece la función, seguramente tardará mucho.

In [44]:
import time
from sklearn.metrics import f1_score, accuracy_score

tini = time.process_time()

from sklearn.model_selection import GridSearchCV
clf = GridSearchCV(estimator=svm.SVC(), param_grid={'C': [2,3,4,5,10],
                                                    'kernel': ('linear', 'rbf', 'poly', 'sigmoid')})
clf.fit(X_train, y_train)

#Obtener los mejores hiper-parámetros
print(clf.best_params_)
#Obtener el mejor clasificador
print(clf.best_estimator_)

tfin = time.process_time()
print()
print("Ha tardado", round(tfin-tini, 2), "segundos")

{'C': 10, 'kernel': 'rbf'}
SVC(C=10)

Ha tardado 212.64 segundos


No ha tardado tanto al final. El mejor modelo es el que lleva `kernel = 'rbf'` y `C=10`. Ese valor de C es un poco sospechoso, en el sentido de que puede estar provocando un sobreajuste, pero lo probaremos igual. A continuación, ajustamos este modelo y vemos su evaluación:

#### Modelo óptimo de SVM según GrindSearchCV validado mediante Holdout

In [10]:
import time
from sklearn.metrics import f1_score, accuracy_score

tini = time.process_time()

clf = svm.SVC(kernel='rbf', C=10, random_state=42).fit(X_train, y_train)
y_pred = clf.predict(X_test)

a=f1_score(y_test, y_pred, average=None)
print("F1", a)

a=f1_score(y_test, y_pred, average='macro') #Calculate metrics for each label, and find their unweighted mean. 
                                                #This does not take label imbalance into account.
print("macro F1", a)

a=f1_score(y_test, y_pred, average='micro') #Calculate metrics globally by counting the total true positives, 
                                                #false negatives and false positives.
print("micro F1", a)

print()
print('Métricas con validación cruzada: ')

macro = cross_val_score(clf, X_train, y_train, cv=5, scoring='f1_macro')
print("macro F1", macro)

micro = cross_val_score(clf, X_train, y_train, cv=5, scoring='f1_micro')
print("micro F1", micro)

print("Accuracy:", accuracy_score(y_test, y_pred, normalize=True))

tfin = time.process_time()
print()
print("Ha tardado", round(tfin-tini, 2), "segundos")

F1 [0.88477952 0.89480519]
macro F1 0.8897923555818293
micro F1 0.890020366598778

Métricas con validación cruzada: 
macro F1 [0.87292852 0.85837528 0.88996044 0.84544183 0.85893987]
micro F1 [0.87330317 0.85859729 0.89027149 0.84615385 0.85956965]
Accuracy: 0.890020366598778

Ha tardado 18.08 segundos


En efecto, conseguimos unos buenos resultados.

### Modelo óptimo de regresión logística según RandomizedSearchCV

Por ver también resultados de RandomizedSearchCV, probaremos la regresión logística.

In [50]:
import time
tini = time.process_time()

logistic = LogisticRegression()
distributions = dict(C=uniform(loc=0, scale=4), penalty=['l2', 'l1'])
clf = RandomizedSearchCV(logistic, distributions)
clf.fit(X_train, y_train)
print(clf.best_params_)
print(clf.best_estimator_)

tfin = time.process_time()
print()
print("Ha tardado", round(tfin-tini, 2), "segundos")

{'C': 0.7839693033701929, 'penalty': 'l2'}
LogisticRegression(C=0.7839693033701929)

Ha tardado 1.25 segundos


C:\Users\guill\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
35 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
35 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\guill\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\guill\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "C:\Users\guill\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 449, in _check_solver
    % (solver, penalty)
Va

In [88]:
import time
from sklearn.metrics import f1_score, accuracy_score

tini = time.process_time()

clf = LogisticRegression(C = 0.7839693033701929, penalty = 'l2').fit(X_train, y_train)
y_pred = clf.predict(X_test)

a=f1_score(y_test, y_pred, average=None)
print("F1", a)

a=f1_score(y_test, y_pred, average='macro') #Calculate metrics for each label, and find their unweighted mean. 
                                                #This does not take label imbalance into account.
print("macro F1", a)

a=f1_score(y_test, y_pred, average='micro') #Calculate metrics globally by counting the total true positives, 
                                                #false negatives and false positives.
print("micro F1", a)

print()
print('Métricas con validación cruzada: ')

macro = cross_val_score(clf, X_train, y_train, cv=5, scoring='f1_macro')
print("macro F1", macro)

micro = cross_val_score(clf, X_train, y_train, cv=5, scoring='f1_micro')
print("micro F1", micro)

tfin = time.process_time()
print()
print("Ha tardado", round(tfin-tini, 2), "segundos")

F1 [0.5472561  0.63647491]
macro F1 0.591865502880855
micro F1 0.5967413441955194

Métricas con validación cruzada: 
macro F1 [0.59884488 0.61521295 0.57724463 0.60636132 0.59602528]
micro F1 [0.60180995 0.62556561 0.58710407 0.6199095  0.60815402]

Ha tardado 0.75 segundos


Los resultados son iguales a lo que sacamos en la anterior sesión.

#### Modelo óptimo de regresión logística según RandomizedSearchCV validado mediante Holdout

In [58]:
import time
from sklearn.metrics import f1_score, accuracy_score

tini = time.process_time()

logistic = LogisticRegression(C=0.7839693033701929, penalty = 'l2').fit(X_train, y_train)
y_pred = logistic.predict(X_test)

a=f1_score(y_test, y_pred, average=None)
print("F1", a)

a=f1_score(y_test, y_pred, average='macro') #Calculate metrics for each label, and find their unweighted mean. 
                                                #This does not take label imbalance into account.
print("macro F1", a)

a=f1_score(y_test, y_pred, average='micro') #Calculate metrics globally by counting the total true positives, 
                                                #false negatives and false positives.
print("micro F1", a)

print()
print('Métricas con validación cruzada: ')

macro = cross_val_score(logistic, X_train, y_train, cv=5, scoring='f1_macro')
print("macro F1", macro)

micro = cross_val_score(logistic, X_train, y_train, cv=5, scoring='f1_micro')
print("micro F1", micro)


tfin = time.process_time()
print()
print("Ha tardado", round(tfin-tini, 2), "segundos")

F1 [0.5472561  0.63647491]
macro F1 0.591865502880855
micro F1 0.5967413441955194

Métricas con validación cruzada: 
macro F1 [0.59884488 0.61521295 0.57724463 0.60636132 0.59602528]
micro F1 [0.60180995 0.62556561 0.58710407 0.6199095  0.60815402]

Ha tardado 0.47 segundos


Muy rápido, pero con resultados notablemente peores a la máquina de soporte vectorial.

### GridSearchCV Red neuronal

In [7]:
import time
from sklearn.neural_network import MLPClassifier

tini = time.process_time()

parameters = {'solver': ['lbfgs'], 'max_iter': [1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000 ], 'alpha': 10.0 ** -np.arange(1, 10), 'hidden_layer_sizes':np.arange(10, 15), 'random_state':[0,1,2,3,4,5,6,7,8,9]}
clf = GridSearchCV(MLPClassifier(), parameters, n_jobs=-1)
clf.fit(X_train, y_train)
pred = clf.predict(X_eval)
score = f1_score(y_eval, pred, average='macro')

print("Score: ",score)
tfin = time.process_time()
print()
print("Ha tardado", round(tfin-tini, 2), "segundos")

KeyboardInterrupt: 

## K-fold

### K-fold con el modelo SVM óptimo anterior

Vamos a seguir la misma estructura que hemos llevado en Holdout, pero validando ahora el modelo óptimo SVM de antes con validación cruzada: **K-fold**. Vamos a hacer un bucle con distintos valores de k.

**Nota**: No aplicaremos el método Leave one-out porque tenemos 5892 observaciones, lo que significaría ajustar 5892 modelos.

In [70]:
import time
from sklearn.model_selection import KFold

tini = time.process_time()

for i in range(3,11):
    kf = KFold(n_splits=i)
    print('K =',i)    
    f=[]
    mac=[]
    mic=[]
    for train, test in kf.split(X_train):
        X_train_i, X_test_i, y_train_i, y_test_i = X_train[train], X_train[test], y_train[train], y_train[test]

        clf = svm.SVC(kernel='rbf', C=10, random_state=42).fit(X_train_i, y_train_i)
        y_pred_i = clf.predict(X_test_i)
        
        f1=f1_score(y_test_i, y_pred_i, zero_division=1)
        f.append(f1)

        macro=f1_score(y_test_i, y_pred_i, average='macro') #Calculate metrics for each label, and find their unweighted mean. 
                                                        #This does not take label imbalance into account.
        mac.append(macro)

        micro=f1_score(y_test_i, y_pred_i, average='micro') #Calculate metrics globally by counting the total true positives, 
                                                        #false negatives and false positives.
        mic.append(micro)
    print("F1 promedio", sum(f)/i)
    print("macro F1 promedio", sum(mac)/i)
    print("micro F1 promedio", sum(mic)/i)
    print()


tfin = time.process_time()
print()
print("Ha tardado", round(tfin-tini, 2), "segundos")

K = 3
F1 promedio 0.8592180928412723
macro F1 promedio 0.8513284521556909
micro F1 promedio 0.8517764200045259

K = 4
F1 promedio 0.864896356257197
macro F1 promedio 0.8569168593221425
micro F1 promedio 0.857432659518657

K = 5
F1 promedio 0.8728889909618672
macro F1 promedio 0.8655765420297232
micro F1 promedio 0.8660318330660081

K = 6
F1 promedio 0.8756994031635278
macro F1 promedio 0.8680161061462147
micro F1 promedio 0.8685177742709378

K = 7
F1 promedio 0.8791617304648592
macro F1 promedio 0.8718311145323209
micro F1 promedio 0.8723717054074734

K = 8
F1 promedio 0.8777240171134087
macro F1 promedio 0.8702411378016688
micro F1 promedio 0.8707871425950676

K = 9
F1 promedio 0.8740722717933121
macro F1 promedio 0.8666036979320474
micro F1 promedio 0.867164516859018

K = 10
F1 promedio 0.8798470596839125
macro F1 promedio 0.8732499091013217
micro F1 promedio 0.8737238485137645


Ha tardado 79.92 segundos


Los resultados de cada fold casi no varían. A partir de K=5 parece que se queda en una pequeña mejora, pero, de nuevo, casi insignificante. Por tanto, podemos reafirmar que el modelo óptimo según GrindSearchCV es bueno, ya que no es sensible a diferentes muestras.

## Stratified K-fold

### Stratified K-fold con el modelo SVM óptimo

Simplemente por probar lo mostrado en el boletín de la práctica, vamos a aplicar Stratified K-fold con nuestro modelo de SVM.

In [73]:
import time
from sklearn.model_selection import StratifiedKFold

tini = time.process_time()

for i in range(3,11):
    kf = StratifiedKFold(n_splits=i)
    print('K =',i)    
    f=[]
    mac=[]
    mic=[]
    #Para hacer las particiones es necesario usar la variable con las clases (y_train)
    for train, test in kf.split(X_train,y_train):
        X_train_i, X_test_i, y_train_i, y_test_i = X_train[train], X_train[test], y_train[train], y_train[test]

        clf = svm.SVC(kernel='rbf', C=10, random_state=42).fit(X_train_i, y_train_i)
        y_pred_i = clf.predict(X_test_i)
        
        f1=f1_score(y_test_i, y_pred_i, zero_division=1)
        f.append(f1)

        macro=f1_score(y_test_i, y_pred_i, average='macro') #Calculate metrics for each label, and find their unweighted mean. 
                                                        #This does not take label imbalance into account.
        mac.append(macro)

        micro=f1_score(y_test_i, y_pred_i, average='micro') #Calculate metrics globally by counting the total true positives, 
                                                        #false negatives and false positives.
        mic.append(micro)
    print("F1 promedio", sum(f)/i)
    print("macro F1 promedio", sum(mac)/i)
    print("micro F1 promedio", sum(mic)/i)
    print()


tfin = time.process_time()
print()
print("Ha tardado", round(tfin-tini, 2), "segundos")

K = 3
F1 promedio 0.8595567502422353
macro F1 promedio 0.8520630507936539
micro F1 promedio 0.8524553066304593

K = 4
F1 promedio 0.8650489487636854
macro F1 promedio 0.8567057203293226
micro F1 promedio 0.8572064151747656

K = 5
F1 promedio 0.8727838511540004
macro F1 promedio 0.8651291861998027
micro F1 promedio 0.8655790881558652

K = 6
F1 promedio 0.8754381589566088
macro F1 promedio 0.8675718979195913
micro F1 promedio 0.8680657974947397

K = 7
F1 promedio 0.880317426174198
macro F1 promedio 0.8725487491560031
micro F1 promedio 0.873049824756479

K = 8
F1 promedio 0.8769384612138575
macro F1 promedio 0.8694077316731261
micro F1 promedio 0.8698821644783394

K = 9
F1 promedio 0.8768726478231097
macro F1 promedio 0.8696857758430473
micro F1 promedio 0.8701063589047298

K = 10
F1 promedio 0.8811554020228973
macro F1 promedio 0.8742105847283919
micro F1 promedio 0.8746267737864377


Ha tardado 87.12 segundos


Resultados similares a los de K-fold.

## Árboles de decisión

Vamos a repetir todo (lo más importante) con árboles de decisión, que era el segundo modelo con mejor puntuación de lo que probé en la anterior sesión de esta práctica. A ver si vemos alguna mejora.

### Modelo óptimo obtenido con GrindSearchCV

Los valores a probar del argumento max_depth son aleatorios. Lo mejor sería sacar una lista con valores de uno en uno, pero tardaría demasiado.

In [84]:
import time
from sklearn.tree import DecisionTreeClassifier

tini = time.process_time()

tree_para = {'criterion':['gini','entropy'],'max_depth':[4,5,6,7,8,9,10,11,12,15,20,30,40,50,70,90,120,150]}
clf = GridSearchCV(DecisionTreeClassifier(), tree_para, cv=5).fit(X_train, y_train)

print(clf.best_params_)
print(clf.best_estimator_)

tfin = time.process_time()
print()
print("Ha tardado", round(tfin-tini, 2), "segundos")

{'criterion': 'gini', 'max_depth': 30}
DecisionTreeClassifier(max_depth=30)

Ha tardado 195.47 segundos


In [12]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(criterion = 'gini', max_depth=30).fit(X_train, y_train)
y_pred = clf.predict(X_test)

a=f1_score(y_test, y_pred, average=None)
print("F1", a)

f1=f1_score(y_test, y_pred, zero_division=1)
print(f1)

macro=f1_score(y_test, y_pred, average='macro')
print(macro)

micro=f1_score(y_test, y_pred, average='micro')
print(micro)

print("Accuracy:", accuracy_score(y_test, y_pred, normalize=True))

F1 [0.73257468 0.75584416]
0.7558441558441559
0.7442094178936284
0.7447386286490155
Accuracy: 0.7447386286490156


### Validación con K-fold

In [86]:
import time
from sklearn.model_selection import KFold

tini = time.process_time()

for i in range(3,11):
    kf = KFold(n_splits=i)
    print('K =',i)    
    f=[]
    mac=[]
    mic=[]
    for train, test in kf.split(X_train):
        X_train_i, X_test_i, y_train_i, y_test_i = X_train[train], X_train[test], y_train[train], y_train[test]

        clf = DecisionTreeClassifier(criterion = 'gini', max_depth=30).fit(X_train_i, y_train_i)
        y_pred_i = clf.predict(X_test_i)
        
        f1=f1_score(y_test_i, y_pred_i, zero_division=1)
        f.append(f1)

        macro=f1_score(y_test_i, y_pred_i, average='macro') #Calculate metrics for each label, and find their unweighted mean. 
                                                        #This does not take label imbalance into account.
        mac.append(macro)

        micro=f1_score(y_test_i, y_pred_i, average='micro') #Calculate metrics globally by counting the total true positives, 
                                                        #false negatives and false positives.
        mic.append(micro)
    print("F1 promedio", sum(f)/i)
    print("macro F1 promedio", sum(mac)/i)
    print("micro F1 promedio", sum(mic)/i)
    print()


tfin = time.process_time()
print()
print("Ha tardado", round(tfin-tini, 2), "segundos")

K = 3
F1 promedio 0.7322523633286195
macro F1 promedio 0.7231317515872545
micro F1 promedio 0.7234668477031003

K = 4
F1 promedio 0.7406224217171944
macro F1 promedio 0.7279967890372572
micro F1 promedio 0.7286678634664568

K = 5
F1 promedio 0.7283298888464206
macro F1 promedio 0.7197437270910065
micro F1 promedio 0.720299472694383

K = 6
F1 promedio 0.7375698017569682
macro F1 promedio 0.7260656327938594
micro F1 promedio 0.7266320939767565

K = 7
F1 promedio 0.7375583286252215
macro F1 promedio 0.7253275245965852
micro F1 promedio 0.7261823564307065

K = 8
F1 promedio 0.7362605760133003
macro F1 promedio 0.7254447824353656
micro F1 promedio 0.7259587035144273

K = 9
F1 promedio 0.7337889172532734
macro F1 promedio 0.7211703789929348
micro F1 promedio 0.7221090744512334

K = 10
F1 promedio 0.741218776130362
macro F1 promedio 0.7328272487802739
micro F1 promedio 0.7334190086290927


Ha tardado 70.86 segundos


De nuevo el valor de K no afecta a la puntuación. Estas puntuaciones son casi iguales a las de la anterior sesión, que daban 0.71, pero al menos hay un poco de mejora. Por último, vamos a ver qué sacamos con Stratified K-fold.

### Validación con Stratified K-fold

In [87]:
import time
from sklearn.model_selection import StratifiedKFold

tini = time.process_time()

for i in range(3,11):
    kf = StratifiedKFold(n_splits=i)
    print('K =',i)    
    f=[]
    mac=[]
    mic=[]
    #Para hacer las particiones es necesario usar la variable con las clases (y_train)
    for train, test in kf.split(X_train,y_train):
        X_train_i, X_test_i, y_train_i, y_test_i = X_train[train], X_train[test], y_train[train], y_train[test]

        clf = DecisionTreeClassifier(criterion = 'gini', max_depth=30).fit(X_train_i, y_train_i)
        y_pred_i = clf.predict(X_test_i)
        
        f1=f1_score(y_test_i, y_pred_i, zero_division=1)
        f.append(f1)

        macro=f1_score(y_test_i, y_pred_i, average='macro') #Calculate metrics for each label, and find their unweighted mean. 
                                                        #This does not take label imbalance into account.
        mac.append(macro)

        micro=f1_score(y_test_i, y_pred_i, average='micro') #Calculate metrics globally by counting the total true positives, 
                                                        #false negatives and false positives.
        mic.append(micro)
    print("F1 promedio", sum(f)/i)
    print("macro F1 promedio", sum(mac)/i)
    print("micro F1 promedio", sum(mic)/i)
    print()


tfin = time.process_time()
print()
print("Ha tardado", round(tfin-tini, 2), "segundos")

K = 3
F1 promedio 0.7372196254686901
macro F1 promedio 0.7241547931605288
micro F1 promedio 0.724824620954967

K = 4
F1 promedio 0.7400164599601524
macro F1 promedio 0.7281211114124765
micro F1 promedio 0.7286688881238115

K = 5
F1 promedio 0.7392397739704394
macro F1 promedio 0.7262178521962313
micro F1 promedio 0.7268603024448738

K = 6
F1 promedio 0.7426056285596392
macro F1 promedio 0.7320254639060538
micro F1 promedio 0.7325154735217195

K = 7
F1 promedio 0.7328349071734294
macro F1 promedio 0.7186289147113973
micro F1 promedio 0.7193868339528233

K = 8
F1 promedio 0.7415215441834929
macro F1 promedio 0.7304388287713256
micro F1 promedio 0.730932397725188

K = 9
F1 promedio 0.7449113880979565
macro F1 promedio 0.7333095133459523
micro F1 promedio 0.7338764426340801

K = 10
F1 promedio 0.7485861140709582
macro F1 promedio 0.7388610133996587
micro F1 promedio 0.7393039266988847


Ha tardado 68.53 segundos


Los resultados se ven incluso menos afectados por las diferentes muestras, llegando a mejorarse un poco más incluso las métricas (muy poco, alrededor de 1%).

## Breves conclusiones 

GrindSearchCV ha sido un buen descubrimiento que usaré en el futuro, me habría gustado saberlo desde antes. El *ranking* de los modelos se mantiene igual que en la sesión anterior de esta práctica.